In [2]:
from pymongo import MongoClient
import json

database_string = 'dungeons'
client = MongoClient('mongodb://localhost:27017/')
db = client[database_string]

Los datos de hints de obtienen mediante:
```
db.rooms.aggregate([
  {
    $unwind: "$hints"
  },
  {
    $project: {
	“_id”: 0,
      "Creation_date": "$hints.creation_date",
      "HintText": "$hints.hintText",
      "Category": "$hints.category",
      "References_room": {
        "IdR": "$room_id",
        "Name": "$room_name",
        "IdD": "$dungeon_id",
        "Dungeon": "$dungeon_name"
      },
      "Publish_by": {
        "Email": "$hints.publish_by.email",
        "User_name": "$hints.publish_by.user_name",
        "CreationDate": "$hints.publish_by.creation_date",
        "Country": "$hints.publish_by.country"
      }
    }
  }
])
```

Seguidamente borramos hints de rooms y users:

```
db.rooms.updateMany(
   {},
   { $unset: { "hints": 1} }
)

db.users.updateMany(
   {},
   { $unset: { "hints": 1} }
)

```

## Cambios en las consultas comentadas

### POST /comment: 
Añade un nuevo comentario
- Parametros: user_email (str), room_id (int), text (str), category (str).

In [6]:
from datetime import datetime

def post_comment(user_email, room_id, category, text):
   try:

        room_info = db.rooms.find_one({'room_id': room_id}, {'_id': 0, 'room_id': 1, 'room_name': 1, 'dungeon_id':1, 'dungeon_name':1})
        user_info = db.users.find_one({'email': user_email}, {'_id': 0, 'email': 1, 'user_name': 1, 'country': 1, 'creation_date': 1})
        user = user_info['user_name']
        country = user_info['country']
        creation_date = user_info['creation_date']

        dungeon_name = room_info['dungeon_name']
        dungeon_id = room_info['dungeon_id']
        room_name = room_info['room_name']
        

        db.Hints.insert_one(
            {
                "Creation_date": datetime.now(),
                "HintText": text,
                "Category": category,
                "References_room": {
                    "idR": room_id,
                    "Name": room_name,
                    "idD": dungeon_id,
                    "Dungeon": dungeon_name
                },
                "Publish_by": {
                    "Email": user_email,
                    "User_name": user,
                    "CreationDate": creation_date,
                    "Country": country
                }
            }
        )
      

        return "Comment added successfully"
   except Exception as e:
        return "Error adding comment: " + str(e)

In [4]:
print(list(db.Hints.find({'Publish_by.Email': 'aaoki@example.com'})))

[]


In [7]:
post_comment('aaoki@example.com', 1, 'lore', 'This is a test comment')

'Comment added successfully'

In [8]:
print(list(db.Hints.find({'Publish_by.Email': 'aaoki@example.com'})))

[{'_id': ObjectId('664789d1ce9bb98446820c58'), 'Creation_date': datetime.datetime(2024, 5, 17, 18, 46, 9, 210000), 'HintText': 'This is a test comment', 'Category': 'lore', 'References_room': {'idR': 1, 'Name': 'pantry of otaku', 'idD': 0, 'Dungeon': 'Burghap, Prison of the Jealous Hippies'}, 'Publish_by': {'Email': 'aaoki@example.com', 'User_name': 'rikamurakami', 'CreationDate': '2021-01-06', 'Country': 'ja_JP'}}]


### GET /dungeon/{dungeon_id}: 

Devuelve información sobre una mazmorra del juego. 
Debe devolver: idM, name y lore. Además, este endpoint se utiliza para alimentar un grafo interactivo
por lo que requiere la siguiente información: 
1. el nombre e id de cada habitación de la
mazmorra
2. las conexiones entre habitaciones de la mazmorra 
3. el id y el nombre de los
monstruos que aparecen en cada habitación
4. el id y el nombre de los tesoros que aparecen en
cada habitación
5. el número de comentarios de cada categoría que hay en cada habitación.

In [14]:
def get_dungeon_by_id(id):
    response = {}
    query_dungeon = db.rooms.find_one({'dungeon_id': id}, {'_id': 0, 'dungeon_id': 1, 'dungeon_name': 1})
    response['dungeon_id'] = query_dungeon['dungeon_id']
    response['dungeon_name'] = query_dungeon['dungeon_name']
    response['rooms'] = []

    rooms = db.rooms.find({'dungeon_id': id}, {'_id': 0, 'room_id': 1, 'room_name': 1, 'rooms_connected': 1, 'in_waypoint': 1, 'out_waypoint': 1}).sort('room_id', 1)

    lore = db.Hints.aggregate([
        {"$match": {"References_room.IdD": id}},
        {"$match": {"Category": "lore"}},
        {"$project": {"_id": 0, "IdR": "$References_room.IdR", "text": "$HintText"}}
    ])
    response['lore'] = list(lore)

    for room in rooms:
        room_info = {
            'room_id': room['room_id'],
            'room_name': room['room_name'],
            'rooms_connected': room['rooms_connected'],
            'hints_distribution': []
        }

        # Obtener el número de comentarios de cada categoría en la habitación
        count_of_comment_categories = db.Hints.aggregate([
            {"$match": {"References_room.IdR": room['room_id']}},
            {"$group": {"_id": "$Category", "count": {"$sum": 1}}},
            {"$project": {"_id": 0, "category": "$_id", "count": 1}}
        ])
        room_info['hints_distribution'] = list(count_of_comment_categories)

        # extraer lista de monsters
        monsters = db.monster.find({'in_rooms.room_id': room['room_id']}, {'_id': 0, 'id': 1, 'name': 1}).sort('id', 1)
        room_info['monsters'] = list(monsters)

        # extraer lista de loot
        loot = db.loot.find({'in_rooms.room_id': room['room_id']}, {'_id': 0, 'id': 1, 'name': 1}).sort('id', 1)
        room_info['loot'] = list(loot)

        # también se puede realizar así, pero dado que hay mas rooms que monsters y para disminuir complejidad, se decidió hacer la primera opción
        # monsters = db.rooms.aggregate([
        #     {"$match": {"room_id": room['room_id']}},
        #     {"$unwind": "$monsters"},
        #     {"$lookup": {
        #         "from": "monster",
        #         "localField": "monsters.id",
        #         "foreignField": "id",
        #         "as": "monster_info"
        #     }},
        #     {"$unwind": "$monster_info"},
        #     {"$project": {"_id": 0, "id": "$monster_info.id", "name": "$monster_info.name"}}
        # ])
        
        response['rooms'].append(room_info)

    return json.dumps(response, indent=4)

In [16]:
print(get_dungeon_by_id(1))

{
    "dungeon_id": 1,
    "dungeon_name": "Burgstream, Culverts of the Bashful Sumo Wrestlers",
    "rooms": [
        {
            "room_id": 33,
            "room_name": "drawing room of unknowns",
            "rooms_connected": [
                {
                    "room_id": 45,
                    "room_name": "great hall of rogues"
                }
            ],
            "hints_distribution": [
                {
                    "count": 1,
                    "category": "hint"
                },
                {
                    "count": 5,
                    "category": "lore"
                },
                {
                    "count": 3,
                    "category": "bug"
                }
            ],
            "monsters": [],
            "loot": []
        },
        {
            "room_id": 34,
            "room_name": "hall ",
            "rooms_connected": [
                {
                    "room_id": 35,
                    "room_name"

### GET /room/{room_id}: 
Devuelve la siguiente información de una habitación: idR, name, inWP y
outWP. Además, incluye la siguiente información de todos los monstruos incluidos en la
habitación (idM, name, type, level, place, exp, manPage) y de los tesoros (idL, name, type1,
type2, weight, gold). Por último, incluye todos los comentarios que se hayan realizado para esa
habitación, cada comentario debe incluir: email y userName, country, creationDate del usuario
que lo realizo y texto, fecha de publicación y categoría del comentario.

In [21]:
def get_room_by_id(id):
    query = db.rooms.find_one({'room_id': id}, 
                          {'_id': 0,
                            'room_id': 1,
                            'room_name': 1, 
                            'in_waypoint': 1, 
                            'out_waypoint': 1, 
                            'monsters.id': 1,
                            'monsters.name': 1,
                            'monsters.type': 1,
                            'monsters.level': 1,
                            'monsters.place': 1, 
                            'monsters.man_page': 1,
                            'loot.id': 1,
                            'loot.name': 1,
                            'loot.gold': 1,
                            'loot.type1': 1,
                            'loot.type2': 1,
                            'loot.weight': 1})
    
    hints = db.Hints.aggregate([
        {"$match": {"References_room.IdR": id}},
        {"$project": 
            {"_id": 0,
            'email': '$Publish_by.Email',
            'userName': '$Publish_by.User_name',
            'country': '$Publish_by.Country',
            'user_CreationDate': '$Publish_by.CreationDate',

            'category': '$Category',
            'text': '$HintText',
            'creation_date': '$Creation_date'
            }
        }
    ])

    query['hints'] = list(hints)
    

    
    return json.dumps(query, indent=4)

In [23]:
print(get_room_by_id(1))

{
    "loot": [
        {
            "id": 233,
            "gold": 5,
            "name": "Padded",
            "type1": "Light Armor",
            "type2": "AC 11",
            "weight": "8"
        },
        {
            "id": 233,
            "gold": 5,
            "name": "Padded",
            "type1": "Light Armor",
            "type2": "AC 11",
            "weight": "8"
        },
        {
            "id": 233,
            "gold": 5,
            "name": "Padded",
            "type1": "Light Armor",
            "type2": "AC 11",
            "weight": "8"
        },
        {
            "id": 233,
            "gold": 5,
            "name": "Padded",
            "type1": "Light Armor",
            "type2": "AC 11",
            "weight": "8"
        },
        {
            "id": 233,
            "gold": 5,
            "name": "Padded",
            "type1": "Light Armor",
            "type2": "AC 11",
            "weight": "8"
        },
        {
            "id": 233,
      

### GET /user/{email}: 
Devuelve todos los campos de un usuario. Además, incluye todos los
comentarios que ha realizado. De cada comentario incluye, el texto, la fecha de creación, la
categoría, el id (Room.IdR) y nombre (Room.name) de la habitación a la que hace referencia el
comentario, el id(Dungeon.IdD) y nombre(Dungeon.name) de la mazmorra donde está la
habitación.

In [24]:
def get_user_by_email(email):
    query = db.users.find_one({'email': email}, 
                          {'_id': 0,
                            'email': 1,
                            'user_name': 1, 
                            'country': 1, 
                            'creation_date': 1,
                            'hints.category': 1,
                            'hints.text': 1,
                            'hints.creation_date': 1,
                            'hints.referemces_room.room_id': 1,
                            'hints.referemces_room.room_name': 1,
                            'hints.referemces_room.dungeon_id': 1,
                            'hints.referemces_room.dungeon_name': 1
                            })
    hints = db.Hints.aggregate([
        {"$match": {"Publish_by.Email": email}},
        {"$project": 
            {"_id": 0,
            'category': '$Category',
            'text': '$HintText',
            'creation_date': '$Creation_date',
            'references_room': '$References_room'
            }
        }
    ])
    query['hints'] = list(hints)

    return json.dumps(query, indent=4)

In [26]:
print(get_user_by_email('lei26@example.org'))

{
    "email": "lei26@example.org",
    "country": "zh_CN",
    "user_name": "nawang",
    "creation_date": "2020-02-02",
    "hints": [
        {
            "category": "suggestion",
            "text": "\u4ecb\u7ecd\u5a01\u671b\u6700\u540e\u5927\u5bb6\u5168\u56fd\u5b66\u6821\u6765\u81ea\u7535\u8bdd.\u7f8e\u56fd\u5168\u56fd\u53d1\u8868\u4f1a\u5458\u5176\u4ed6\u7f51\u7edc\u7cfb\u5217.\\n\u4e1c\u897f\u53d1\u8868\u7b49\u7ea7\u5206\u6790.\u5e94\u8be5\u7279\u522b\u53d1\u5e03.\\n\u4e0d\u8981\u8bbe\u8ba1\u4e00\u4e9b.\u5404\u79cd\u91cd\u8981\u4e00\u4e2a\u5f53\u524d\u56de\u590d\u4eca\u5929.\u5973\u4eba\u5177\u6709\u4e13\u4e1a\u4e2a\u4eba\u5bf9\u4e8e\u8fd9\u4e48.\\n\u63d0\u4f9b\u6765\u6e90\u8bba\u575b\u6e38\u620f\u8054\u7cfb\u6709\u4e9b\u4e4b\u95f4.\u76ee\u524d\u51fa\u73b0\u81ea\u5df1\u56de\u590d\u8d44\u6599.\u624b\u673a\u5de5\u7a0b\u53ea\u6709\u8bf4\u660e\u7c7b\u522b\u7136\u540e.\\n\u901a\u8fc7\u4ee5\u540e\u624b\u673a\u8fd9\u6837\u81ea\u5df1\u65f6\u95f4.\\n\u90e8\u95e8\u5927\u5bb6\u53d1\u5c55